# Sagemaker demo

Sagemaker has 3 options for training and deploying models:
1. A standard Sagemaker model, like xgboost.
2. A custom `sklearn` model using a pre-built sagemaker scikit-learn docker image.
3. Any other custom build model using an own custom image

In this notebook we demonstrate how to do option 1 and 2, which can be done without using a `Dockerfile`. Option 3 involves a bit more engineering.

For part one we start by following this [step-by-step guide](https://aws.amazon.com/getting-started/tutorials/build-train-deploy-machine-learning-model-sagemaker/) on how to model with sagemaker.

In [ ]:
import boto3
import os

bucket_name = 'playbucket-steven'

## Example data

Here we are going to predict Black Friday sales based on this [kaggle data set](https://www.kaggle.com/mehdidag/black-friday/version/1).
- Its around 5 MB
- Its dimensions are 538k x 12

### Getting data from S3

Boto is the AWS SDK for Python. Access S3 using boto:

In [ ]:
# create s3 resource
s3 = boto3.resource('s3')

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)
    # print out file names in bucket, called 'keys'
    for objects in bucket.objects.all():
        print('- ', objects.key)

Amazon S3 does not provide compute, such as zip compression/decompression. You would need to write a program that:
- Downloads the zip file
- Extracts the files
- Does actions on the files

In [ ]:
import pandas as pd
import zipfile

# Download if not there yet
if not os.path.isfile('../data/black-friday.zip'):
    s3.Bucket(bucket_name).download_file('black-friday.zip', '../data/black-friday.zip')

# Extract and read with pandas
zf = zipfile.ZipFile('../data/black-friday.zip') 
df = pd.read_csv(zf.open('BlackFriday.csv'))    

Or just use the super nice pandas API:

In [ ]:
df = pd.read_csv('s3://' + bucket_name + '/black-friday.zip')

### Examine and preprocess dataset

Target column to predict: 	`Purchase` = Purchase amount in dollars

In [ ]:
df.shape

Create X and y data sets

In [ ]:
X = df.iloc[:, 7:] # keep only integer columns to avoid one hot encoding
y = X.pop('Purchase')

Split in train test and create a model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Approach 1: Using a standard `Sagemaker` model

To prepare the data, train the machine learning model, and deploy it, you will need to import some libraries and define a few environment variables in your Jupyter notebook environment. 

In [ ]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

To use a SageMaker pre-built XGBoost model, you will need to reformat the header and first column of the training data and load the data from the S3 bucket. 

Using [Sagemaker XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html). Also check out this [example XGBoost notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/xgboost_abalone/xgboost_abalone.ipynb)

For CSV training, the algorithm assumes that the target variable is in the first column and that the CSV does not have a header record. For CSV inference, the algorithm assumes that CSV input does not have the label column.



In [ ]:
# write training data to csv in the required XGBoost format
pd.concat([y_train, X_train], axis=1).to_csv('./data/train.csv', index=False, header=False)

# upload to bucket used for training
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('./data/train.csv')

# load training data for sagemaker
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


Next, you need to set up the SageMaker session, create an instance of the XGBoost model (an estimator), and define the model’s hyperparameters. 

Note, for a regression task you will need to set `objective='reg:linear'`.

In [ ]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='reg:linear',num_round=100)  # set objective function here

With the data loaded and XGBoost estimator set up, train the model using gradient optimization on a ml.m4.xlarge instance by executing the `fit` command.

This kicks off a training job (see UI), where the training happens. Does not happen on this notebook instance!

In [ ]:
xgb.fit({'train': s3_input_train})

### Deploy the model

You will deploy the trained model to an endpoint, reformat then load the CSV data, then run the model to create predictions.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m5.large')

Making a prediction

In [ ]:
# test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).as_matrix() #load the data into an array
X_test_array = X_test.as_matrix()

xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type

predictions = xgb_predictor.predict(X_test_array).decode('utf-8') # predict!

predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

### Evaluate your model

In this step, you will evaluate the performance and accuracy of the machine learning model.


In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, predictions_array, multioutput='variance_weighted')

### Terminate your resources

To delete the SageMaker endpoint and possibly the objects in your S3 bucket

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

# bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
# bucket_to_delete.objects.all().delete()

## Approach 2: Using a custom `sklearn` model

Let's first develop a custom model, and grid search pipeline.

In [ ]:
from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV


pipeline = Pipeline([
    ('imputer', Imputer()),
    ('regressor', DecisionTreeRegressor())
])


params = {'regressor__max_depth': [2, 3, 4, 5]}

# replace len(data) for n depending on target
grid_search = GridSearchCV(pipeline, 
                           n_jobs=-1,
                           param_grid=params, 
                           cv=5)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.predict(X_test)

In [ ]:
grid_search.score(X_test, y_test)

### Now what?

We have a model, how do we deploy it and make an endpoint?

Use the pre-build sklearn image as is done in this [tutorial](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb)

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'pipeline.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.m5.large",
    role=role,
    sagemaker_session=sess,
    hyperparameters={'regressor_max_depth': 4})  # hyperparameters that are not tuned

Probably the above is for running just a standard model. Now I have also put grid search in the `pipeline.py`, but this might be removed and should be part of the hyperparameter tuning that sagemaker offers through `HyperParameterTrainingJobs`, see [here](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex-tuning-job.html)

In [ ]:
# train_input = sess.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY) )

# sklearn.fit({'train': train_input})
sklearn.fit({'train': s3_input_train})

#### Deploy the model 

Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count and instance type.

In [ ]:
sklearn_predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m5.large")

In [ ]:
# same as for the xgboost inference job

sklearn_predictor.content_type = 'text/csv' # set the data type for an inference
sklearn_predictor.serializer = csv_serializer # set the serializer type

sklearn_predictions = sklearn_predictor.predict(X_test_array)#.decode('utf-8') # predict!

sklearn_predictions_array = np.fromstring(sklearn_predictions[1:], sep=',') # and turn the prediction into an array
print('predictions shape:', sklearn_predictions_array.shape)

print('r2_score:', r2_score(y_test, predictions_array, multioutput='variance_weighted'))

Done.